In [1]:
import sqlite3

# Use only the first time
conn = sqlite3.connect("mydatabase.db") # or use :memory: to put it in RAM
cursor = conn.cursor()


In [2]:
# Read keywords to track from track.csv into variable trackstring

import csv
with open('track.csv', newline='') as csvfile:
    trackreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in trackreader:
        trackstring = ', '.join(row)
        print(','.join(row))

        print

BMW, daimler, mercedes benz, AMG, audi


In [14]:
# The main body of the streamer class, writes to SQL

from twython import TwythonStreamer  
import json

# Filter out unwanted data
def process_tweet(tweet):  
    
    # Tweet Data: (8 items)
    # tweet_log_id, tweet_id_text, tweet_hashtag , tweet_text, created_at,  geo_lat,  geo_long , user_id_text
    d = ["1", tweet['id_str'], "test", tweet['text'], tweet['created_at'], tweet['user']['id_str']]
    

    # User Data: (11 items) 
    # user_id_text, screen_name, name, location, url,  description text,
    # created_at, followers_count ,  friends_count ,statuses_count , time_zone 
    d2 = [tweet['user']['id_str'], tweet['user']['screen_name'], tweet['user']['created_at']]
    
    # Tweet log data: (6 items)
    # tweet_log_id, query text, geo_lat, geo_long, radius, timestamp
    d3 = []
    
    return [d, d2,d3]


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer): 
    
    # The class constructor
    def __init__(self,app_key, app_secret, oauth_token, oauth_token_secret,queryText='',long='',lat='',Radius=''):
        TwythonStreamer.__init__(self,app_key, app_secret, oauth_token, oauth_token_secret)
        self.queryText = queryText
        self.long = long
        self.lat = lat
        self.Radius = Radius

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            print('query info:',self.queryText,self.lat,self.long)
            tweet_data = process_tweet(data)
            user_id = data['user']['id_str']
            self.save_to_sql(tweet_data, user_id)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_sql(self, tweet, user_id):
        print(tweet)
        
        
        #cursor.execute("SELECT count(*) FROM user WHERE user_id_text=?", (user_id,))
        
        # Save the User Data, if the user is Unique
       # if cursor.fetchone()[0] == 0:
       #     cursor.execute("INSERT INTO user(user_id_text, screen_name, created_at) VALUES (?,?,?)", tweet[1])
        
        # Save the Tweet Data
       # cursor.execute("INSERT INTO tweet(tweet_log_id,tweet_id_text,tweet_hashtag,tweet_text,created_at,user_id_text) VALUES (?,?,?,?,?,?)", tweet[0])
        
        # Save the Tweet log data
        
        #conn.commit()

In [15]:
# Get the credentials:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN_KEY'], creds['ACCESS_TOKEN_SECRET'])
# Start the stream
#geo_locations = '-122.75,36.8,-121.75,37.8,-74,40,-73,41'
#stream.statuses.filter(track=trackstring,locations=geo_locations)  
stream.statuses.filter(track=trackstring)  

query info:   
[['1', '1021707625975750656', 'test', '@IEAffiliate @Brian_Kitchener @CollinOctantis @keigh_see @nicholaspitts @Col_Sandurz @badibulgator @alastairjallen1… https://t.co/8DaHX8uAvQ', 'Tue Jul 24 10:44:15 +0000 2018', '896072194714771456'], ['896072194714771456', 'RunningHippo', 'Fri Aug 11 18:13:54 +0000 2017']]
query info:   
[['1', '1021707644078354432', 'test', 'RT @GVLeasing: 🔥BMW Offer🔥\n\n520d M Sport Saloon Auto\n\n6+35\n\n10k miles per year\n\nBCH or PCH\n\n£305.99 ex vat per month \n\nOther contract terms…', 'Tue Jul 24 10:44:19 +0000 2018', '710419306807496704'], ['710419306807496704', 'BlackpoolRockz', 'Thu Mar 17 10:55:36 +0000 2016']]
query info:   
[['1', '1021707653368754176', 'test', 'RT @JeannieG40: Justice of the Peace: Do you take this man to be your lawfully wedded husband?\n\nMe: Audi.', 'Tue Jul 24 10:44:21 +0000 2018', '970459949276061696'], ['970459949276061696', '_steamy_mac', 'Mon Mar 05 00:44:16 +0000 2018']]
query info:   
[['1', '1021707686226

KeyboardInterrupt: 